#### Solution 생성 및 Versioning
솔루선 생성 및 하이퍼 파라미터 조정

In [23]:
import boto3
from botocore.exceptions import ClientError
import json
import time


personalize = boto3.client("personalize")

#### 솔루션 생성

In [2]:
# recipe (알고리즘) 선택
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

In [4]:
# 이미 dataset group이 생성되어 있을 때, 조회
# dataset_group_name = "osungmart-dataset-groupmwomk"
dataset_group_name = "osungmart-dataset-group20230204"

list_dataset_group_resp = personalize.list_dataset_groups(
  maxResults=10
)

for dataset_grp in list_dataset_group_resp['datasetGroups']:
  if dataset_grp['name'] == dataset_group_name:
    dataset_group_arn = dataset_grp['datasetGroupArn']
    print("dataset_group_arn: {}".format(dataset_grp['datasetGroupArn']) )

dataset_group_arn: arn:aws:personalize:ap-northeast-2:491818659652:dataset-group/osungmart-dataset-group20230204


In [5]:
# create solution
solution_name = "osugmart-user-personalize-solution-HPO"
try: 
  create_solution_resp = personalize.create_solution(
    name = solution_name,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
    performHPO=True
  )
  solution_arn = create_solution_resp['solutionArn']
  print(json.dumps(create_solution_resp, indent=2))
except ClientError as e:
  print(e)

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:491818659652:solution/osugmart-user-personalize-solution-HPO",
  "ResponseMetadata": {
    "RequestId": "84d36286-6df1-4467-97ac-b24ccc62b139",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 04 Feb 2023 13:02:14 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "113",
      "connection": "keep-alive",
      "x-amzn-requestid": "84d36286-6df1-4467-97ac-b24ccc62b139"
    },
    "RetryAttempts": 0
  }
}


In [24]:
solution_arn = "arn:aws:personalize:ap-northeast-2:491818659652:solution/osugmart-user-personalize-solution-HPO"

In [25]:
# Create Solution Version: training and retraining
try:
  create_solution_version_resp = personalize.create_solution_version(
    solutionArn = solution_arn
  )
  solution_version_arn = create_solution_version_resp['solutionVersionArn']
  print(json.dumps(create_solution_version_resp, indent=2))
except ClientError as e:
  print(e)


{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:491818659652:solution/osugmart-user-personalize-solution-HPO/90a54ee1",
  "ResponseMetadata": {
    "RequestId": "ab8832c4-b5c7-48a5-85fc-a7093e3b3a86",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 08 Feb 2023 00:33:38 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "129",
      "connection": "keep-alive",
      "x-amzn-requestid": "ab8832c4-b5c7-48a5-85fc-a7093e3b3a86"
    },
    "RetryAttempts": 0
  }
}


**Solution Versioning은 많은 시간이 소요되므로, 모니터링은 Console에서 확인하는 것이 좋다**

In [7]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS


KeyboardInterrupt: 

#### 성능 평가
AWS User Personalization metrics

In [10]:
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client("personalize-events")

user insteraction 2건 이상, 10 만건은 metric이 매우 좋지 않았음

In [21]:
# 2023. 02. 07. 2nd. user interaction: 2건, 100만건
solution_version_arn = "arn:aws:personalize:ap-northeast-2:491818659652:solution/osugmart-user-personalize-solution-HPO/afda062c"

In [22]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)
print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:491818659652:solution/osugmart-user-personalize-solution-HPO/afda062c",
  "metrics": {
    "coverage": 0.0345,
    "mean_reciprocal_rank_at_25": 0.2693,
    "normalized_discounted_cumulative_gain_at_10": 0.3098,
    "normalized_discounted_cumulative_gain_at_25": 0.3242,
    "normalized_discounted_cumulative_gain_at_5": 0.2959,
    "precision_at_10": 0.0362,
    "precision_at_25": 0.0168,
    "precision_at_5": 0.0644
  },
  "ResponseMetadata": {
    "RequestId": "e9b9abb7-80f5-4dc1-a66d-5283e3831671",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 08 Feb 2023 00:09:33 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "427",
      "connection": "keep-alive",
      "x-amzn-requestid": "e9b9abb7-80f5-4dc1-a66d-5283e3831671"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
# 2023. 02. 08. 3th. user interactin: 5, 200만건
solution_version_arn = ""

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)
print(json.dumps(get_solution_metrics_response, indent=2))